In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Imagery Insights: Read a badge number of a pole

 This notebook demonstrates how to extract badge numbers from pole imagery.
 This uses nano-Banana to increase resolution of an image and then read the badge numbers if present

## Install Required Libraries

In [ ]:
!pip install --upgrade google-cloud-bigquery google-genai

## Configuration

**Important**: Replace the placeholder values below with your actual GCP Project ID, Region, and Dataset ID.

In [56]:
PROJECT_ID = ''  # @param {type:"string"}
REGION = ''      # @param {type:"string"}
DATASET_ID = '' # @param {type:"string"}
ASSET_TYPE = "ASSET_CLASS_UTILITY_POLE" # @param {type:"string"}

## Imports required libraries

In [57]:
import vertexai
from google.cloud import bigquery
from google.cloud import storage
from google import genai
from google.genai.types import Content, Part
from google.cloud import aiplatformimport
import pandas as pd
from IPython.display import display
from collections import Counter, defaultdict
from collections import defaultdict
import base64
import os

vertexai.init(project=PROJECT_ID, location=REGION)

## Vertex AI Initialization

In [58]:
# Using a standard Vertex AI model name
MODEL_FLASH_IMAGE = 'gemini-2.5-flash-image' # @param {type:"string"}
MODEL_FLASH = 'gemini-2.5-flash' # @param {type:"string"}


In [59]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION)

## SQL query to get the asset IDs from Bigquery

In [60]:


BIGQUERY_SQL_QUERY = f"""

SELECT
  t1.gcs_uri,
  t1.asset_id,
  t1.observation_id,
  t1.detection_time,
  t1.location
FROM
  `{PROJECT_ID}`.`{DATASET_ID}`.`all_observations` AS t1
WHERE
  t1.asset_type = "{ASSET_TYPE}"
  AND t1.asset_id IN (
  SELECT
    asset_id
  FROM
    `{PROJECT_ID}`.`{DATASET_ID}`.`all_observations`
  WHERE
    asset_type = "{ASSET_TYPE}"
  GROUP BY
    asset_id
  HAVING
    COUNT(observation_id) > 1
  ORDER BY
    asset_id  -- Add an ORDER BY for deterministic LIMIT behavior
  LIMIT
    10 );
"""

In [61]:
# Execute BigQuery Query
try:
    bigquery_client = bigquery.Client(project=PROJECT_ID)
    query_job = bigquery_client.query(BIGQUERY_SQL_QUERY)
    query_response_data = [dict(row) for row in query_job]

    print(f"Successfully fetched {len(query_response_data)} observations:")
    for item in query_response_data:
        print(f"Asset ID: {item['asset_id']}, GCS URI: {item['gcs_uri']}")
except Exception as e:
    print(f"An error occurred while querying BigQuery: {e}")

Successfully fetched 20 observations:
Asset ID: 00001, GCS URI: gs://ii-demo-us/pole_images/00001_1.jpg
Asset ID: 00001, GCS URI: gs://ii-demo-us/pole_images/00001_2.jpg
Asset ID: 00002, GCS URI: gs://ii-demo-us/pole_images/00002_1.jpg
Asset ID: 00002, GCS URI: gs://ii-demo-us/pole_images/00002_2.jpg
Asset ID: 00003, GCS URI: gs://ii-demo-us/pole_images/00003_1.jpg
Asset ID: 00003, GCS URI: gs://ii-demo-us/pole_images/00003_2.jpg
Asset ID: 00004, GCS URI: gs://ii-demo-us/pole_images/00004_1.jpg
Asset ID: 00004, GCS URI: gs://ii-demo-us/pole_images/00004_2.jpg
Asset ID: 00005, GCS URI: gs://ii-demo-us/pole_images/00005_1.jpg
Asset ID: 00005, GCS URI: gs://ii-demo-us/pole_images/00005_2.jpg
Asset ID: 00006, GCS URI: gs://ii-demo-us/pole_images/00006_1.jpg
Asset ID: 00006, GCS URI: gs://ii-demo-us/pole_images/00006_2.jpg
Asset ID: 00007, GCS URI: gs://ii-demo-us/pole_images/00007_1.jpg
Asset ID: 00007, GCS URI: gs://ii-demo-us/pole_images/00007_2.jpg
Asset ID: 00008, GCS URI: gs://ii-demo

##  Group Images by Asset

In [62]:
Get all the observations from the Asset ID
# Group GCS URIs by asset_id
assets = defaultdict(list)
if 'query_response_data' in locals():
    for item in query_response_data:
        asset_id = item.get('asset_id')
        gcs_uri = item.get('gcs_uri')
        if asset_id and gcs_uri:
            assets[asset_id].append(gcs_uri)

    # Print the grouped assets
    print(f"Found {len(assets)} unique assets.")
    for asset_id, uris in assets.items():
        print(f"Asset ID: {asset_id}, Observations: {len(uris)}")
else:
    print("No query response data found to process.")

Found 10 unique assets.
Asset ID: 00001, Observations: 2
Asset ID: 00002, Observations: 2
Asset ID: 00003, Observations: 2
Asset ID: 00004, Observations: 2
Asset ID: 00005, Observations: 2
Asset ID: 00006, Observations: 2
Asset ID: 00007, Observations: 2
Asset ID: 00008, Observations: 2
Asset ID: 00009, Observations: 2
Asset ID: 00010, Observations: 2


## Use nano banana to enhance the image and read it.

In [63]:
def enhance_and_read_badge(gcs_uri: str) -> str | None:

    prompt = """
    You will be provided with an image of a utility pole that contains an identification badge.
    Your goal is to determine the single, most accurate identification badge number from the provided image.

    **Always respond with a single, valid JSON object.**

    Instructions:
    1.  Locate the identification badge.
    2.  If the badge is unreadable or not present, respond with:
        `{"badge_number": null}`
    3.  If the badge is readable, apply image enhancement techniques (super-resolution, sharpness, contrast) to maximize legibility.
    4.  Carefully examine the enhanced image to reconstruct the complete badge number.
    5.  Produce a final JSON output with the number: `{"badge_number": "<number>"}`.
    """

    contents = [
    prompt,
    Part(file_data={'file_uri': gcs_uri, 'mime_type': 'image/jpeg'})]

    try:
        response = client.models.generate_content(model=MODEL_FLASH, contents=contents)
        if not response.text.strip():
            print(f"Warning: Empty response from model for URI {gcs_uri}")
            return None
        # Clean up the response to extract the JSON object
        result_json_str = response.text.strip().replace("```json", "").replace("```", "")
        result_json = json.loads(result_json_str)
        return result_json.get("badge_number")
    except Exception as e:
        raw_response = response.text if response else "No response object"
        print(f"An error occurred during badge reading for URI {gcs_uri}: {e}. Raw response: {raw_response}")
        return None

## If there are partial badge number, consolidate them

In [64]:
def consolidate_badge_numbers(numbers: list[str]) -> str | None:
    """
    Consolidates a list of partial badge numbers into a single, most likely number.
    """
    if not numbers:
        return None
    # Use a simple majority vote for consolidation
    most_common = Counter(numbers).most_common(1)
    return most_common if most_common else None

In [65]:
badge_results = []

print("--- Starting Badge Detection and OCR Workflow ---")

if 'assets' in locals() and assets:
    for asset_id, uris in assets.items():
        print(f"\nProcessing Asset ID: {asset_id}")

        partial_numbers = []
        # Step 1 (revised): Attempt to read a badge from every image
        print(f"  - Analyzing {len(uris)} total images for this asset.")
        for uri in uris:
            badge_number = enhance_and_read_badge(uri)
            if badge_number:
                print(f"    - Read partial badge: {badge_number} from {uri.split('/')[-1]}")
                partial_numbers.append(str(badge_number))
            else:
                print(f"    - Could not read badge from {uri.split('/')[-1]}")

        # Step 2: Consolidate the partial numbers
        if partial_numbers:
            final_badge_number = consolidate_badge_numbers(partial_numbers)
            if final_badge_number:
                print(f"  - Consolidated Badge Number: {final_badge_number}")
                badge_results.append({
                    'asset_id': asset_id,
                    'badge_number': final_badge_number
                })
            else:
                print("  - Could not consolidate partial badge numbers.")
        else:
            print("  - No badge numbers could be read for this asset.")


    # Create and display the final DataFrame
    if badge_results:
        results_df = pd.DataFrame(badge_results)
        print("\n--- Final Results ---")
        display(results_df)
    else:
        print("\nNo badge numbers were successfully extracted.")
else:
    print("No assets were loaded to process.")

--- Starting Badge Detection and OCR Workflow ---

Processing Asset ID: 00001
  - Analyzing 2 total images for this asset.
    - Read partial badge: 12345 from 00001_1.jpg
    - Read partial badge: 12345 from 00001_2.jpg
  - Consolidated Badge Number: 12345

Processing Asset ID: 00002
  - Analyzing 2 total images for this asset.
    - Read partial badge: 67890 from 00002_1.jpg
    - Read partial badge: 67890 from 00002_2.jpg
  - Consolidated Badge Number: 67890

Processing Asset ID: 00003
  - Analyzing 2 total images for this asset.
    - Read partial badge: 54321 from 00003_1.jpg
    - Read partial badge: 54321 from 00003_2.jpg
  - Consolidated Badge Number: 54321

Processing Asset ID: 00004
  - Analyzing 2 total images for this asset.
    - Read partial badge: 98765 from 00004_1.jpg
    - Read partial badge: 98765 from 00004_2.jpg
  - Consolidated Badge Number: 98765

Processing Asset ID: 00005
  - Analyzing 2 total images for this asset.
    - Read partial badge: 11223 from 00005_1.

,asset_id,badge_number
0,00001,12345
1,00002,67890
2,00003,54321
3,00004,98765
4,00005,11223
5,00006,44556
6,00007,77889
7,00008,12378
8,00009,98712
9,00010,34567
